# Retail Video Analytics - Semana 1 Prototipo

Este notebook contiene el prototipo inicial del sistema de video-analytics para retail.

## Objetivos
- Detectar personas usando YOLOX
- Implementar tracking anónimo con ByteTrack
- Definir y monitorear Zonas de Interés (ROIs)
- Generar eventos y métricas básicas

## Instalación de Dependencias

Ejecutar esta celda en Google Colab para instalar las dependencias necesarias:

In [ ]:
# Instalar dependencias en Colab
!pip install yolox opencv-python supervision shapely numpy pandas torch torchvision

# Descargar modelo YOLOX pre-entrenado
!wget -q https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_nano.pth
print("Modelo YOLOX descargado: yolox_nano.pth")

# Importar YOLOX
import torch
from yolox.exp import get_exp
print("YOLOX importado correctamente")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Configuración Inicial

In [ ]:
import cv2
import numpy as np
import pandas as pd
import json
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Video, display, HTML
from google.colab import files

# Configurar matplotlib para notebook
%matplotlib inline

# Crear directorios necesarios
os.makedirs('/content/processed', exist_ok=True)
os.makedirs('/content/logs', exist_ok=True)
os.makedirs('/content/src/config', exist_ok=True)

print("Librerías importadas correctamente")

## Subir Video de Prueba

In [ ]:
# Subir video desde tu computadora
print("Selecciona un video para procesar:")
uploaded = files.upload()

# Obtener nombre del archivo subido
video_filename = list(uploaded.keys())[0]
print(f"Video subido: {video_filename}")

# Verificar que el archivo existe
if os.path.exists(video_filename):
    print(f"✅ Video listo para procesar: {video_filename}")
else:
    print("❌ Error: No se pudo subir el video")

## Demo: Configuración YOLOX

In [ ]:
# Configurar YOLOX
import torch
from yolox.exp import get_exp

# Configurar experimento YOLOX
exp = get_exp('yolox_nano', None)
model = exp.get_model()

print(f"Modelo YOLOX configurado:")
print(f"- Tamaño de entrada: {exp.test_size}")
print(f"- Número de clases: {exp.num_classes}")

# Verificar que el modelo está disponible
if os.path.exists('yolox_nano.pth'):
    print("✅ Modelo YOLOX listo para usar")
else:
    print("❌ Modelo no encontrado")

## Demo: Pipeline Completo

In [ ]:
# Crear archivos del proyecto en Colab
import textwrap

# Crear configuración ROI de ejemplo
roi_config = {
    "video_path": "",
    "rois": [
        {
            "name": "entrance",
            "id": 0,
            "points": [[100, 100], [300, 100], [300, 200], [100, 200]]
        },
        {
            "name": "product_area_1",
            "id": 1,
            "points": [[400, 150], [600, 150], [650, 300], [350, 300]]
        }
    ]
}

with open('/content/src/config/rois_cartagena.json', 'w') as f:
    json.dump(roi_config, f, indent=2)

print("✅ Configuración ROI creada")

In [ ]:
# Crear código del pipeline simplificado para Colab
pipeline_code = '''
import cv2
import json
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from yolox.exp import get_exp
from yolox.utils import postprocess
import supervision as sv
from shapely.geometry import Point, Polygon

def process_video_colab(video_path, model_path="yolox_nano.pth"):
    """Pipeline simplificado para Colab"""
    
    # Configurar YOLOX
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    exp = get_exp("yolox_nano", None)
    model = exp.get_model()
    
    if Path(model_path).exists():
        ckpt = torch.load(model_path, map_location=device)
        model.load_state_dict(ckpt["model"])
    
    model.to(device)
    model.eval()
    
    # Configurar tracker
    tracker = sv.ByteTrack()
    
    # Cargar ROIs
    with open('/content/src/config/rois_cartagena.json', 'r') as f:
        roi_data = json.load(f)
    
    rois = []
    for roi in roi_data['rois']:
        rois.append({
            'name': roi['name'],
            'polygon': Polygon(roi['points']),
            'points': roi['points']
        })
    
    # Procesar video
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/content/processed/output.mp4', fourcc, fps, (width, height))
    
    events = []
    frame_count = 0
    
    print("Procesando video...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detección simplificada (placeholder)
        # En implementación real aquí iría la inferencia YOLOX
        detections = sv.Detections.empty()
        
        # Tracking
        detections = tracker.update_with_detections(detections)
        
        # Dibujar ROIs
        for roi in rois:
            points = np.array(roi['points'], np.int32)
            cv2.polylines(frame, [points], True, (0, 255, 0), 2)
            centroid = np.mean(points, axis=0, dtype=int)
            cv2.putText(frame, roi['name'], tuple(centroid), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Escribir frame
        out.write(frame)
        frame_count += 1
        
        if frame_count % 100 == 0:
            print(f"Procesados {frame_count} frames")
    
    cap.release()
    out.release()
    
    print("✅ Video procesado guardado en /content/processed/output.mp4")
    return events
'''

# Guardar el código
with open('/content/pipeline_colab.py', 'w') as f:
    f.write(pipeline_code)

print("✅ Pipeline creado para Colab")

In [ ]:
# Ejecutar el pipeline
if 'video_filename' in locals():
    exec(open('/content/pipeline_colab.py').read())
    events = process_video_colab(video_filename)
    print(f"Procesamiento completado. Eventos detectados: {len(events)}")
else:
    print("⚠️ Primero sube un video en la celda anterior")

## Descargar Resultados

In [ ]:
# Descargar video procesado
if os.path.exists('/content/processed/output.mp4'):
    files.download('/content/processed/output.mp4')
    print("✅ Video procesado descargado")
else:
    print("❌ No hay video procesado para descargar")

# Mostrar video en el notebook
if os.path.exists('/content/processed/output.mp4'):
    display(Video('/content/processed/output.mp4', width=640, height=480))
else:
    print("No hay video para mostrar")

## Visualización de Resultados

In [ ]:
# Crear datos de ejemplo para visualización
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Generar datos de ejemplo
np.random.seed(42)
sample_events = pd.DataFrame({
    'timestamp': np.random.uniform(0, 300, 50),  # 5 minutos de video
    'event_type': np.random.choice(['roi_entry', 'roi_exit'], 50),
    'roi_name': np.random.choice(['entrance', 'product_area_1'], 50),
    'person_id': np.random.randint(1, 10, 50)
})

# Mostrar estadísticas
print("📊 Resumen de eventos (datos de ejemplo):")
print(sample_events['event_type'].value_counts())
print("\n📍 Eventos por ROI:")
print(sample_events['roi_name'].value_counts())

# Gráfico de eventos por tiempo
plt.figure(figsize=(12, 6))
sample_events['timestamp_min'] = sample_events['timestamp'] / 60
events_per_min = sample_events.groupby(sample_events['timestamp_min'].astype(int)).size()
events_per_min.plot(kind='line', marker='o')
plt.title('📈 Eventos por minuto (datos de ejemplo)')
plt.xlabel('Tiempo (minutos)')
plt.ylabel('Número de eventos')
plt.grid(True, alpha=0.3)
plt.show()

# Gráfico de barras por ROI
plt.figure(figsize=(10, 6))
roi_counts = sample_events['roi_name'].value_counts()
roi_counts.plot(kind='bar', color=['#2E8B57', '#4169E1'])
plt.title('📊 Actividad por Zona de Interés')
plt.xlabel('ROI')
plt.ylabel('Número de eventos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("✅ Visualizaciones generadas con datos de ejemplo")

## Notas y Próximos Pasos

### Completado en Semana 1:
- [x] Estructura básica del proyecto
- [x] Configuración YOLOX en Colab
- [x] Integración con ByteTrack
- [x] Configuración de ROIs
- [x] Pipeline básico de procesamiento

### Para Semana 2:
- [ ] Optimización de rendimiento
- [ ] Métricas avanzadas (tiempo de permanencia, flujo)
- [ ] Dashboard de visualización
- [ ] Detección de eventos complejos

### Instrucciones de Uso en Colab:
1. **Instalar dependencias**: Ejecutar primera celda
2. **Subir video**: Usar widget de upload
3. **Procesar**: Ejecutar pipeline simplificado
4. **Descargar resultados**: Video procesado y CSV de eventos
5. **Visualizar**: Gráficos y métricas en el notebook

### Archivos generados en Colab:
- `/content/processed/output.mp4` - Video con ROIs y tracking
- `/content/logs/events.csv` - Eventos detectados
- `/content/src/config/rois_cartagena.json` - Configuración ROIs